In [ ]:
import geopandas
import re
import pandas as pd
import folium
import requests
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [ ]:
# API Call Function
def call_api(url):
    lta_api_key = os.getenv("LTA_API_KEY")
    if not lta_api_key:
        raise ValueError("API key not found. Please set the LTA_API_KEY environment variable.")

    # Define endpoint to call
    endpoint = url

    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialise the output list to hold the results
    output = []

    # Loop through the pages to get all the results
    while (len(output) % 500) == 0:
        # Set the params to skip to the correct page
        params = {"$skip": len(output)}

        # Make the GET request
        req = requests.get(endpoint, headers=headers, params=params)

        if req.status_code != 200:
            raise Exception(f"Error fetching data: {req.text}")

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)

In [ ]:
# Bus Services

bus_services = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusServices")
bus_services

In [ ]:
# Bus Services

bus_services = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusServices")
bus_services

In [ ]:
# Bus Routes

bus_routes = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes")
bus_routes

In [ ]:
# Bus Stops

bus_stops = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusStops")
bus_stops

In [ ]:
# Bus Routes with Bus Stop Descriptions

bus_route_names = pd.merge(bus_routes, bus_stops, on = 'BusStopCode', how = 'left')
bus_route_names

In [ ]:
# Bus Stop Codes & Coordinates

bus_stops_coordinates = bus_stops[['BusStopCode', 'Latitude', 'Longitude']]
bus_stops_coordinates['BusStopCode'] = bus_stops_coordinates['BusStopCode'].astype(int)

bus_stops_coordinates